In [ ]:
# !pip install -q \
#     pandas numpy \
#     sentence-transformers \
#     faiss-cpu \
#     openai \
#     langgraph langchain \
#     pydantic \
#     ipywidgets


In [ ]:
# !pip install -q "pydantic-ai-slim[openrouter]" nest_asyncio tqdm

In [2]:
import pandas as pd
import numpy as np
import faiss
import pickle
from sentence_transformers import SentenceTransformer, CrossEncoder
from pydantic import BaseModel
from typing import Optional, List
import os
import nest_asyncio
nest_asyncio.apply()

from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openrouter import OpenRouterModel
from pydantic_ai.providers.openrouter import OpenRouterProvider

import ipywidgets as widgets
from IPython.display import display, clear_output


In [ ]:
OPENROUTER_API_KEY = ""
os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY

MODEL_NAME = "qwen/qwen3-30b-a3b-instruct-2507"

provider = OpenRouterProvider(api_key=OPENROUTER_API_KEY)

def get_model(model_name: str = MODEL_NAME) -> OpenRouterModel:
    return OpenRouterModel(
        model_name=model_name,
        provider=provider
    )

print(f"Используем модель: {MODEL_NAME}")


Используем модель: qwen/qwen3-30b-a3b-instruct-2507


In [28]:
class SearchIntent(BaseModel):
    query: str = Field(description="Что именно ищет пользователь")
    category: str | None = Field(description="Категория описываемого товара")
    max_price: int | None = Field(default=None)
    region: str | None = Field(default=None)
    city: str | None = Field(default=None)
    condition: str | None = Field(
        default=None,
        description="новый или б/у"
    )


In [29]:
baseline_agent = Agent(
    model=get_model(),
    output_type=SearchIntent,
    system_prompt="""
    Ты AI-ассистент маркетплейса (Авито).

    Твоя задача — извлечь параметры поиска из запроса пользователя.

    Правила:
    - Если параметр не указан — верни null
    - Цена — только число
    - Регион и город разделяй
    - Ничего не выдумывай
    """,
            retries=4
)


In [30]:
async def parse_intent(query: str) -> SearchIntent:
    result = await baseline_agent.run(
        query
    )
    return result.output

def parse_intent_sync(query: str) -> SearchIntent:
    return baseline_agent.run_sync(query).output


In [ ]:
# df = pd.read_csv("train.csv")

# df = df[
#     df["title"].notna() &
#     df["description"].notna() &
#     df["price"].notna()
# ].reset_index(drop=True)

# print("Объявлений:", len(df))


Объявлений: 1305348


In [ ]:
# def build_search_text(row):
#     parts = [
#         row["title"],
#         row["description"],
#         f"Регион: {row['region']}",
#         f"Город: {row['city']}",
#         f"Категория: {row['parent_category_name']} / {row['category_name']}"
#     ]

#     for p in ["param_1", "param_2", "param_3"]:
#         if pd.notna(row[p]):
#             parts.append(f"Характеристика: {row[p]}")

#     return "\n".join(parts)

# df["search_text"] = df.apply(build_search_text, axis=1)


In [ ]:

# import pandas as pd
# import faiss
# import numpy as np
# import pickle
# from sentence_transformers import SentenceTransformer

# df = pd.read_csv("train.csv")

# df = df[
#     df["title"].notna() &
#     df["description"].notna() &
#     df["price"].notna()
# ]

# df["search_text"] = df.apply(build_search_text, axis=1)

# embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# vectors = embedder.encode(
#     df["search_text"].tolist(),
#     batch_size=32,
#     normalize_embeddings=True,
#     show_progress_bar=True
# )

# index = faiss.IndexFlatIP(vectors.shape[1])
# index.add(vectors)

# faiss.write_index(index, "avito.faiss")

# with open("avito_meta.pkl", "wb") as f:
#     pickle.dump(df.to_dict(orient="records"), f)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\User\anaconda3\envs\MTS\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/40793 [00:00<?, ?it/s]

In [72]:
import faiss, pickle
import numpy as np
from sentence_transformers import SentenceTransformer

index = faiss.read_index("avito.faiss")
with open("avito_meta.pkl", "rb") as f:
    META = pickle.load(f)

embedder = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
from typing import Optional
from pydantic import BaseModel, Field, validator
from datetime import datetime
import math
import asyncio
from typing import Optional
from pydantic import BaseModel

class ItemFull(BaseModel):
    item_id: str
    user_id: Optional[str]

    region: Optional[str]
    city: Optional[str]

    parent_category_name: Optional[str]
    category_name: Optional[str]

    param_1: Optional[str] = None
    param_2: Optional[str] = None
    param_3: Optional[str] = None

    title: str
    description: Optional[str]

    price: Optional[float]

    item_seq_number: Optional[int]
    activation_date: Optional[str]

    user_type: Optional[str]

    image: Optional[str]
    image_top_1: Optional[float]

    deal_probability: Optional[float]

    search_text: Optional[str]

    semantic_score: Optional[float]
    
def clean(value):
    if value is None:
        return None
    if isinstance(value, float) and math.isnan(value):
        return None
    return value


def retrieve(intent: SearchIntent, k: int = 200) -> list[ItemFull]:
    q_emb = embedder.encode([intent.query])
    faiss.normalize_L2(q_emb)

    scores, idxs = index.search(q_emb, k)
    items = []

    for score, idx in zip(scores[0], idxs[0]):
        row = META[idx]  # dict

        # --- фильтры ---
        # if intent.city and row.get("city") != intent.city:
        #     continue
        # if intent.region and row.get("region") != intent.region:
        #     continue
        # if intent.max_price and row.get("price") and row["price"] > intent.max_price:
        #     continue

        kwargs = {
            "item_id": str(row.get("item_id")),
            "user_id": clean(row.get("user_id")),

            "region": clean(row.get("region")),
            "city": clean(row.get("city")),

            "parent_category_name": clean(row.get("parent_category_name")),
            "category_name": clean(row.get("category_name")),

            "param_1": clean(row.get("param_1")),
            "param_2": clean(row.get("param_2")),
            "param_3": clean(row.get("param_3")),

            "title": clean(row.get("title")),
            "description": clean(row.get("description")),

            "price": clean(row.get("price")),

            "item_seq_number": clean(row.get("item_seq_number")),
            "activation_date": clean(row.get("activation_date")),

            "user_type": clean(row.get("user_type")),

            "image": clean(row.get("image")),
            "image_top_1": clean(row.get("image_top_1")),

            "deal_probability": clean(row.get("deal_probability")),

            "search_text": clean(row.get("search_text")),

            "semantic_score": float(score),
        }

        item = ItemFull(**kwargs)
        items.append(item)

    return items


In [175]:
class RankedItem(ItemFull):
    reason: Optional[str] = None
    rank_score: Optional[float] = None

    
rerank_agent = Agent(
    model=get_model(),
    system_prompt="""
    Ты ранкер товаров маркетплейса.
    Оцени, насколько товар подходит запросу пользователя.
    Верни число от 0 до 10 и краткую причину.
    """
    )


In [176]:
async def rerank_one(item: ItemFull, intent: SearchIntent):
    prompt = f"""
    Ты — AI-ассистент для поиска товаров (Avito).

    Запрос пользователя:
    {intent.query}

    Товар:
    Название: {item.title}
    Описание: {item.description}
    Категория: {item.parent_category_name} / {item.category_name}
    Цена: {item.price} ₽
    Город: {item.city}
    Регион: {item.region}

    Ответь СТРОГО:
    score: <0-10>
    reason: <коротко>
    """

    try:
        result = await rerank_agent.run(prompt)
        text = result.output
    except Exception as e:
        print(f"[rerank] error {item.item_id}: {e}")
        return None

    score_match = re.search(r"score:\s*([\d.]+)", text, re.I)
    reason_match = re.search(r"reason:\s*(.*)", text, re.I)

    if not score_match:
        return None

    return {
        "item": item,
        "rank_score": float(score_match.group(1)),
        "reason": reason_match.group(1).strip() if reason_match else None
    }

async def rerank_async(
    intent: SearchIntent,
    items: List[ItemFull],
    top_k: int = 10
) -> List[RankedItem]:

    tasks = [
        rerank_one(item, intent)
        for item in items[:10]
    ]

    results = await asyncio.gather(*tasks)

    ranked = []
    for r in results:
        if r is None:
            continue

        item = r["item"]

        base_data = item.dict()
        base_data.setdefault("category", item.category_name or "")
        base_data.setdefault("score", item.semantic_score or 0.0)

        ranked.append(
            RankedItem(
                **base_data,
                rank_score=r["rank_score"],
                reason=r["reason"]
            )
        )

    ranked.sort(key=lambda x: x.rank_score, reverse=True)
    return ranked[:top_k]



In [189]:
explain_agent = Agent(
    model=get_model(),
        system_prompt="""
Ты — помощник маркетплейса (Avito).

Твоя задача — объяснять пользователю, почему подобранные товары ему подходят.

Правила:
- Пиши простым, разговорным языком
- Обращайся к пользователю на "вы"
- Не упоминай ИИ, модель, алгоритмы
- Не повторяй дословно описание товаров
- Не придумывай факты
- Делай акцент на:
  • соответствие запросу
  • цену
  • город / регион
  • категорию
- Будь кратким: 2–4 предложения
- Если вариантов несколько — обобщай, не перечисляй все подряд

Стиль:
дружелюбный, полезный, без воды
"""
)

async def explain(
    intent: SearchIntent,
    items: list[RankedItem]
) -> str:

    bullets = "\n\n".join(
        f"""
        Товар:
        Название: {i.title}
        Описание: {i.description}
        Категория: {i.parent_category_name} / {i.category_name}
        Характеристики: {', '.join([p for p in [i.param_1, i.param_2, i.param_3] if p])}
        Цена: {i.price} ₽
        Город: {i.city}
        Регион: {i.region}
        Релевантность: {i.rank_score}/10
        Почему выбран: {i.reason}
        """.strip()
            for i in items
    )


    prompt = f"""
Запрос пользователя:
{intent.query}

Подобранные товары:
{bullets}

Объясни пользователю, почему эти варианты ему подходят.
"""

    result = await explain_agent.run(prompt)
    return result.output.strip()


In [178]:
REFRASE_PROMPT = """
Ты — AI-ассистент поиска товаров (Avito).

Твоя задача — переписать запрос пользователя так,
чтобы он был МАКСИМАЛЬНО похож на текст,
который используется в поисковом индексе.

Формат индекса:
- Название товара
- Краткое описание
- Регион: <регион>
- Город: <город>
- Категория: <parent> / <category>
- Характеристика: <param>

❗ Правила:
- НЕ добавляй ничего от себя
- НЕ выдумывай параметры
- Используй слова пользователя
- Если регион/город не указан — не добавляй
- Пиши на русском
- Выводи ТОЛЬКО текст, без пояснений

Запрос пользователя:
{query}

Результат:
"""


In [179]:
class RewriteIntent(BaseModel):
    query: str = Field(description="Переписанный запрос.")


In [180]:
from pydantic_ai import Agent

rephrase_agent = Agent(
    model=get_model(),
    system_prompt="Ты переписываешь поисковые запросы под формат поискового индекса."
)

async def rephrase_query(intent: SearchIntent) -> SearchIntent:
    result = await rephrase_agent.run(
        REFRASE_PROMPT.format(query=intent.query)
    )
    
    intent.query = result.output.strip()

    return intent


In [191]:
query = 'Найди консоль PS3 Красноярск'
intent = await parse_intent(query)
rewritten = await rephrase_query(intent)
candidates = retrieve(rewritten)
ranked = await rerank_async(rewritten, candidates)
explanation = await explain(rewritten, ranked)

C:\Users\User\AppData\Local\Temp\ipykernel_37196\3344516468.py:60: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  base_data = item.dict()


In [196]:
async def chat_fn(query: str):
    intent = await parse_intent(query)
    rewritten = await rephrase_query(intent)
    candidates = retrieve(rewritten)
    ranked = await rerank_async(rewritten, candidates)
    print(ranked)
    explanation = await explain(rewritten, ranked)


    lines = []
    for i, item in enumerate(ranked, 1):
        lines.append(
            f"{i}. {item.title} — {int(item.price)} ₽ ({item.city})"
        )

    answer = "🤖 Ави:\n" + "\n".join(lines)
    answer += "\n\nПочему эти варианты:\n" + explanation


    return answer


with gr.Blocks() as demo:
    gr.Markdown("## 🛒 Avito AI Assistant")

    query = gr.Textbox(
        placeholder="детская куртка до 3000 в Екатеринбурге"
    )
    output = gr.Textbox(lines=20)

    btn = gr.Button("Найти")
    btn.click(chat_fn, query, output)

demo.launch()


* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.


C:\Users\User\AppData\Local\Temp\ipykernel_37196\3344516468.py:60: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  base_data = item.dict()


[RankedItem(item_id='9b64f9020191', user_id='fa63b034cf6d', region='Нижегородская область', city='Кстово', parent_category_name='Бытовая электроника', category_name='Игры, приставки и программы', param_1='Игровые приставки', param_2=None, param_3=None, title='Playstation 3', description='продаю так как не играю  , состояние отличное', price=8000.0, item_seq_number=21, activation_date='2017-03-16', user_type='Private', image='04403634ae32276b98b7470b7731381d41f36ffe63ad68e0ab90eac08b5a3ea0', image_top_1=2897.0, deal_probability=0.0, search_text='Playstation 3\nпродаю так как не играю  , состояние отличное\nРегион: Нижегородская область\nГород: Кстово\nКатегория: Бытовая электроника / Игры, приставки и программы\nХарактеристика: Игровые приставки', semantic_score=0.8579355478286743, reason='Соответствует запросу по наименованию, категории и описанию; указана цена, город — релевантно.', rank_score=9.0), RankedItem(item_id='ef0d69e6d2ab', user_id='21e75e64eba6', region='Краснодарский край'